#### Project CCF pySpark RDD

In [2]:
from itertools import islice
g=sc.textFile("FileStore/tables/webGoogleS.txt")
#remove header 4 lines
data= g.mapPartitionsWithIndex(lambda idx, it: islice(it, 4, None) if idx == 0 else it)
data.count()

Out[1]: 41306

In [3]:
pair = data.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1]))
pair.take(5)

Out[2]: [('0', '11342'),
 ('0', '824020'),
 ('0', '867923'),
 ('0', '891835'),
 ('11342', '0')]

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
from pyspark import SparkConf,SparkContext
import os

conf = SparkConf()
conf.setMaster('local')
conf.setAppName('pyspark-shell')
conf.set('spark.driver.host', '127.0.0.1')
conf.set("spark.ui.proxyBase", "")
conf.set("spark.cores.max","4")
conf.set("spark.executor.cores","2")
os.environ['PYSPARK_PYTHON'] = '/Users/ccompain/.pyenv/versions/miniconda3-latest/bin/python' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter' 
sc = SparkContext(conf=conf)

In [10]:
def CCF_dedup(data):
  dedup = data.map(lambda x: ((x[0],x[1]), 1))\
              .reduceByKey(lambda x,y: 1)\
              .map(lambda x: (x[0][0], x[0][1]))
  return dedup

def CCF_Iterate_map(pair):
    #map
    pair2=pair.map(lambda x: (x[1], x[0]))
    map_pair=pair.union(pair2)
    return map_pair

def f(x): return x
    
def CCF_Iterate_reduce(data):
    #find min value per key
    #couple (key, min)
    key_min=data.reduceByKey(lambda x,y: min(x,y)).filter(lambda x: x[0]>x[1])
    #put together: in one command
    valuelist_min=key_min.join(data).map(lambda x: (x[0],x[1][1])).cogroup(key_min).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
    
    #make RDD : (min, (list of value))
    min_valuelist=valuelist_min.map(lambda x:(x[1][1][0], x[1][0]))

    #use flatmapvalue to transform to RDD (min, value), then filter min != value, map to couple (value, min)

    min_value=min_valuelist.flatMapValues(f).filter(lambda x: x[0]!=x[1]).map(lambda x: (x[1],x[0]))
    countnewpair=min_value.count()

    #union couple (key, min) and (value, min)
    unionkeyminvalue=key_min.union(min_value)

    return unionkeyminvalue,min_value
  

In [3]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")])
r.collect()

[('A', 'B'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('F', 'G'), ('G', 'H')]

## Iteration 1

In [4]:
map1= CCF_Iterate_map(r)
map1.collect()

[('A', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('D', 'E'),
 ('F', 'G'),
 ('G', 'H'),
 ('B', 'A'),
 ('C', 'B'),
 ('D', 'B'),
 ('E', 'D'),
 ('G', 'F'),
 ('H', 'G')]

In [15]:
reduce1_tmp,new_pairs=CCF_Iterate_reduce(map1)
reduce1_tmp.collect()

[('C', 'B'),
 ('B', 'A'),
 ('D', 'B'),
 ('G', 'F'),
 ('E', 'D'),
 ('H', 'G'),
 ('H', 'F'),
 ('C', 'A'),
 ('D', 'A'),
 ('E', 'B')]

In [16]:
reduce1=CCF_dedup(reduce1_tmp)
reduce1.collect()

[('B', 'A'),
 ('H', 'F'),
 ('G', 'F'),
 ('D', 'A'),
 ('C', 'A'),
 ('E', 'D'),
 ('D', 'B'),
 ('H', 'G'),
 ('E', 'B'),
 ('C', 'B')]

In [17]:
new_pairs.collect()

[('H', 'F'), ('C', 'A'), ('D', 'A'), ('E', 'B')]

## Iteration 2

In [18]:
map2= CCF_Iterate_map(reduce1)
map2.collect()

[('B', 'A'),
 ('H', 'F'),
 ('G', 'F'),
 ('D', 'A'),
 ('C', 'A'),
 ('E', 'D'),
 ('D', 'B'),
 ('H', 'G'),
 ('E', 'B'),
 ('C', 'B'),
 ('A', 'B'),
 ('F', 'H'),
 ('F', 'G'),
 ('A', 'D'),
 ('A', 'C'),
 ('D', 'E'),
 ('B', 'D'),
 ('G', 'H'),
 ('B', 'E'),
 ('B', 'C')]

In [20]:
reduce2_tmp,new_pairs=CCF_Iterate_reduce(map2)
reduce2_tmp.collect()

[('B', 'A'),
 ('G', 'F'),
 ('D', 'A'),
 ('E', 'B'),
 ('H', 'F'),
 ('C', 'A'),
 ('H', 'F'),
 ('D', 'B'),
 ('G', 'F'),
 ('D', 'A'),
 ('E', 'A'),
 ('C', 'A'),
 ('B', 'A'),
 ('E', 'A'),
 ('B', 'A')]

In [21]:
new_pairs.collect()

[('H', 'F'),
 ('D', 'B'),
 ('G', 'F'),
 ('D', 'A'),
 ('E', 'A'),
 ('C', 'A'),
 ('B', 'A'),
 ('E', 'A'),
 ('B', 'A')]

In [22]:
new_pairs_non_dedup=CCF_dedup(new_pairs)
new_pairs_non_dedup.collect()

[('H', 'F'),
 ('B', 'A'),
 ('C', 'A'),
 ('D', 'A'),
 ('E', 'A'),
 ('G', 'F'),
 ('D', 'B')]

In [23]:
reduce2=CCF_dedup(reduce2_tmp)
reduce2.collect()

[('D', 'A'),
 ('E', 'A'),
 ('G', 'F'),
 ('H', 'F'),
 ('B', 'A'),
 ('C', 'A'),
 ('E', 'B'),
 ('D', 'B')]

## Iteration 3

In [25]:
map3= CCF_Iterate_map(reduce2)
map3.collect()

[('D', 'A'),
 ('E', 'A'),
 ('G', 'F'),
 ('H', 'F'),
 ('B', 'A'),
 ('C', 'A'),
 ('E', 'B'),
 ('D', 'B'),
 ('A', 'D'),
 ('A', 'E'),
 ('F', 'G'),
 ('F', 'H'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'E'),
 ('B', 'D')]

In [26]:
reduce3_tmp,new_pairs=CCF_Iterate_reduce(map3)
reduce3_tmp.collect()

[('D', 'A'),
 ('E', 'A'),
 ('H', 'F'),
 ('B', 'A'),
 ('G', 'F'),
 ('C', 'A'),
 ('B', 'A'),
 ('E', 'A'),
 ('D', 'A'),
 ('B', 'A')]

In [27]:
new_pairs.collect()

[('B', 'A'), ('E', 'A'), ('D', 'A'), ('B', 'A')]

In [30]:
new_pairs_non_dedup=CCF_dedup(new_pairs)
new_pairs_non_dedup.collect()

[('E', 'A'), ('D', 'A'), ('B', 'A')]

In [31]:
reduce3=CCF_dedup(reduce3_tmp)
reduce3.collect()

[('H', 'F'), ('G', 'F'), ('C', 'A'), ('E', 'A'), ('D', 'A'), ('B', 'A')]

## Iteration 4

In [33]:
map4= CCF_Iterate_map(reduce3)
map4.collect()

[('H', 'F'),
 ('G', 'F'),
 ('C', 'A'),
 ('E', 'A'),
 ('D', 'A'),
 ('B', 'A'),
 ('F', 'H'),
 ('F', 'G'),
 ('A', 'C'),
 ('A', 'E'),
 ('A', 'D'),
 ('A', 'B')]

In [34]:
reduce4_tmp,new_pairs=CCF_Iterate_reduce(map4)
reduce4_tmp.collect()

[('D', 'A'), ('E', 'A'), ('H', 'F'), ('B', 'A'), ('G', 'F'), ('C', 'A')]

In [35]:
new_pairs.collect()

[]

In [36]:
reduce4=CCF_dedup(reduce4_tmp)
reduce4.collect()

[('G', 'F'), ('E', 'A'), ('H', 'F'), ('D', 'A'), ('B', 'A'), ('C', 'A')]

## STOP ALGO

In [38]:
print(reduce1.collect())
print(reduce2.collect())
print(reduce3.collect())
print(reduce4.collect())

[('B', 'A'), ('H', 'F'), ('G', 'F'), ('D', 'A'), ('C', 'A'), ('E', 'D'), ('D', 'B'), ('H', 'G'), ('E', 'B'), ('C', 'B')]
[('D', 'A'), ('E', 'A'), ('G', 'F'), ('H', 'F'), ('B', 'A'), ('C', 'A'), ('E', 'B'), ('D', 'B')]
[('H', 'F'), ('G', 'F'), ('C', 'A'), ('E', 'A'), ('D', 'A'), ('B', 'A')]
[('G', 'F'), ('E', 'A'), ('H', 'F'), ('D', 'A'), ('B', 'A'), ('C', 'A')]


# TEST UNITAIRES

In [5]:
key_min=map1.reduceByKey(lambda x,y: min(x,y)).filter(lambda x: x[0]>x[1])
key_min.collect()




[('C', 'B'), ('B', 'A'), ('D', 'B'), ('G', 'F'), ('E', 'D'), ('H', 'G')]

In [6]:
valuelist_min=key_min.join(map1).map(lambda x: (x[0],x[1][1])).cogroup(key_min).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
valuelist_min.collect()

[('C', (['B'], ['B'])),
 ('H', (['G'], ['G'])),
 ('G', (['H', 'F'], ['F'])),
 ('B', (['C', 'D', 'A'], ['A'])),
 ('D', (['E', 'B'], ['B'])),
 ('E', (['D'], ['D']))]

In [7]:
min_valuelist=valuelist_min.map(lambda x:(x[1][1][0], x[1][0]))
#min_valuelist.take(10)
#use flatmapvalue to transform to RDD (min, value), then filter min != value, map to couple (value, min)
def f(x): return x
min_value=min_valuelist.flatMapValues(f).filter(lambda x: x[0]!=x[1]).map(lambda x: (x[1],x[0]))
countnewpair=min_value.count()
min_value.collect()

[('H', 'F'), ('C', 'A'), ('D', 'A'), ('E', 'B')]

In [8]:
unionkeyminvalue=key_min.union(min_value)
unionkeyminvalue.collect()

[('C', 'B'),
 ('B', 'A'),
 ('D', 'B'),
 ('G', 'F'),
 ('E', 'D'),
 ('H', 'G'),
 ('H', 'F'),
 ('C', 'A'),
 ('D', 'A'),
 ('E', 'B')]